In [1]:
import numpy as np
import pandas as pd
import pubchempy as pc
import scipy as sp

#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


## what do i want this model function to do?

I want it to...

1) check to see if the compound is a known enzymatic product
    * if so: return the enzyme/reaction/compound/starting compound
2) if not: pair the input or found compound with all the promiscuous products

3) calculate the distance between the query compound and each product of each promiscuous enzyme (NOT the average distance, just the distance)

4) return the n most similar reactions/substrates/whatever (number n is selected by user, default is hard-coded)

this code needs to do 2-4, 1 is external but needs to talk to this code

inputs: master_df, enzyme_col, pubchem_col, smiles_col, pubchem_cid
need to drop fingerprint and mol from columns if they are there
need to make sure this is the promsicuous master_df (positive only)
this is almost exactly the same inputs as pair_query_compound

____

preparing development dataframe

In [2]:
prom_pos = pd.read_csv('../../../big-datasets/positive-with-dist.csv')

In [3]:
prom_pos.drop(columns=['Fingerprint'], inplace=True)
prom_pos.head()

,enzyme,product,reacts,PubChemID,SMILES,n_C,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW,Dist
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,1.0,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,31,46,2,0,0,0,0,9,450.707,1.000000
1,EC-1.14.14.80,CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,35,4,0,0,0,0,1,315.474,0.714718
2,EC-1.14.14.80,PALMITATE,1.0,504166,CCCCCCCCCCCCCCCC(=O)[O-],16,31,2,0,0,0,0,1,255.422,0.714718
3,EC-1.14.14.80,OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,33,2,0,0,0,0,2,281.460,0.714718
4,EC-1.14.14.80,STEARIC_ACID,1.0,3033836,CCCCCCCCCCCCCCCCCC(=O)[O-],18,35,2,0,0,0,0,1,283.476,0.714718


In [6]:
prom_pos.shape

(7395, 15)

In [4]:
def fingerprint_products(input_df): #fingerprints all products in a given df
    '''DocString'''
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(1, column='Mol', value=mol_list)
    input_df.insert(2, column='Fingerprint', value= fp_list)
            
    return input_df

In [6]:
prom_pos = fingerprint_products(prom_pos)

In [5]:
small = prom_pos.iloc[:100,:].copy()

______

In [5]:
def check_for_known(all_enz_df, mol_fingerprint, threshold):
    
    bad_cols = ['Known', 'Mol', 'Fingerprint']
    
    for col in bad_cols:
        if col in all_enz_df.columns:
            all_enz_df.drop(columns=col, inplace=True)
        else:
            continue
            
    fingerprint = mol_fingerprint
    
    input_df = fingerprint_products(all_enz_df) # fingerprint the input dataframe and return it
    
    input_df['Known'] = '' # initialize similarity column
    
    for index, row in input_df.iterrows():
        similarity = DataStructs.FingerprintSimilarity(fingerprint, row['Fingerprint'], 
                                                       metric=DataStructs.TanimotoSimilarity)
        input_df['Known'].loc[index] = similarity
    
    
    known_df = input_df[input_df['Known'] >= threshold]
    
    if len(known_df) > 0:
        known_df.sort_values(by='Known', ascending=False, inplace=True)
        result = known_df
    else:
        #call to promiscuous search code here
        result = print('No known enzymes. Beginning promiscuous search.')
        
    return result

In [7]:
smile, _ = cid_to_smiles(985)
mol = Chem.rdmolfiles.MolFromSmiles(smile)
fingerprint = FingerprintMols.FingerprintMol(mol)

In [10]:
check_for_known(testing, fingerprint, 0.95)

NameError: name 'testing' is not defined

In [6]:
def cid_to_smiles(cid):

    try:
        compound = pc.get_compounds(cid)[0]
        smiles = compound.canonical_smiles
    except BaseException:
        pass
    
    return smiles, cid


def dist_for_expansion(prom_df, pubchem_cid, num_similar):
    
    #####this probably isn't necessary because in the final version, this will all be run in one sitting and not saved and loaded etc
    #if 'Fingerprint' and 'Mol' in prom_df.columns:
     #   prom_df = prom_df.drop(columns=['Fingerprint','Mol'])
    #else:
     #   continue
    
    bad_cols = ['ExpanDist']
    
    for col in bad_cols:
        if col in prom_df.columns:
            prom_df.drop(columns=col, inplace=True)
        else:
            continue
    
        
    smiles, _ = cid_to_smiles(pubchem_cid)
    
    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    
    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    fingerprint = FingerprintMols.FingerprintMol(mol)
    
    
    prom_df['ExpanDist'] = ''
    
    for index, row in prom_df.iterrows():
        comp = DataStructs.FingerprintSimilarity(row['Fingerprint'], fingerprint, metric=DataStructs.TanimotoSimilarity)
        prom_df['ExpanDist'].loc[index] = comp

    prom_df.sort_values(by='ExpanDist', ascending=False, inplace=True)
    
    if num_similar == 'None':
        n = 20
    else:
        n = num_similar
    
    selected = prom_df.iloc[:n,:].copy()
    
    return selected

In [11]:
smaller = prom_pos.iloc[:20,:].copy()

In [15]:
dist_test = fingerprint_products(testing)

In [17]:
new = dist_for_expansion(dist_test, 985, 5)

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
new.values.tolist()

[['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323,
  0.75],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323,
  0.7377049180327869],
 ['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0,
  0.3515625]]

In [13]:
dist_for_expansion(small, 243, 'None')

KeyError: 'Fingerprint'

In [37]:
%%writefile expansion_models.py
import numpy as np
import pandas as pd
import pubchempy as pc

#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

def check_for_known(all_enz_df, mol_fingerprint, threshold):
    
    bad_cols = ['Known', 'Mol', 'Fingerprint']
    
    for col in bad_cols:
        if col in all_enz_df.columns:
            all_enz_df.drop(columns=col, inplace=True)
        else:
            continue
            
    fingerprint = mol_fingerprint
    
    input_df = fingerprint_products(all_enz_df) # fingerprint the input dataframe and return it
    
    input_df['Known'] = '' # initialize similarity column
    
    for index, row in input_df.iterrows():
        similarity = DataStructs.FingerprintSimilarity(fingerprint, row['Fingerprint'], 
                                                       metric=DataStructs.TanimotoSimilarity)
        input_df['Known'].loc[index] = similarity
    
    
    known_df = input_df[input_df['Known'] >= threshold]
    
    if len(known_df) > 0:
        known_df.sort_values(by='Known', ascending=False, inplace=True)
        result = known_df
    else:
        #call to promiscuous search code here
        result = print('No known enzymes. Beginning promiscuous search.')
        
    return result


def cid_to_smiles(cid):

    try:
        compound = pc.get_compounds(cid)[0]
        smiles = compound.canonical_smiles
    except BaseException:
        pass
    
    return smiles, cid


def dist_for_expansion(prom_df, pubchem_cid, num_similar):
    
    #####this probably isn't necessary because in the final version, this will all be run in one sitting and not saved and loaded etc
    #if 'Fingerprint' and 'Mol' in prom_df.columns:
     #   prom_df = prom_df.drop(columns=['Fingerprint','Mol'])
    #else:
     #   continue
    
    bad_cols = ['ExpanDist']
    
    for col in bad_cols:
        if col in prom_df.columns:
            prom_df.drop(columns=col, inplace=True)
        else:
            continue
    
        
    smiles, _ = cid_to_smiles(pubchem_cid)
    
    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    
    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    fingerprint = FingerprintMols.FingerprintMol(mol)
    
    
    prom_df['ExpanDist'] = ''
    
    for index, row in prom_df.iterrows():
        comp = DataStructs.FingerprintSimilarity(row['Fingerprint'], fingerprint, metric=DataStructs.TanimotoSimilarity)
        prom_df['ExpanDist'].loc[index] = comp

    prom_df.sort_values(by='ExpanDist', ascending=False, inplace=True)
    
    if num_similar == 'None':
        n = 20
    else:
        n = num_similar
    
    selected = prom_df.iloc[:n,:].copy()
    
    return selected

def fingerprint_products(input_df): #fingerprints all products in a given df
    '''DocString'''
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(1, column='Mol', value=mol_list)
    input_df.insert(2, column='Fingerprint', value= fp_list)
            
    return input_df

Overwriting expansion_models.py


In [8]:
testing = prom_pos.iloc[:5,:].copy()

In [16]:
testing.head()

,enzyme,product,reacts,PubChemID,SMILES,n_C,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW,Dist
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,1.0,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,31,46,2,0,0,0,0,9,450.707,1.000000
1,EC-1.14.14.80,CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,35,4,0,0,0,0,1,315.474,0.714718
2,EC-1.14.14.80,PALMITATE,1.0,504166,CCCCCCCCCCCCCCCC(=O)[O-],16,31,2,0,0,0,0,1,255.422,0.714718
3,EC-1.14.14.80,OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,33,2,0,0,0,0,2,281.460,0.714718
4,EC-1.14.14.80,STEARIC_ACID,1.0,3033836,CCCCCCCCCCCCCCCCCC(=O)[O-],18,35,2,0,0,0,0,1,283.476,0.714718


In [9]:
testing = testing[['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist']]

In [18]:
result = check_for_known(testing, fingerprint, 0.95)

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
result.values.tolist()

[['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0]]

In [20]:
testing.values.tolist()

[['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0,
  0.3515625],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323,
  0.7377049180327869],
 ['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323,
  0.75],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0]]

In [12]:
def test_check_for_known():
    
    expected_df = pd.DataFrame([['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist', 'Known'])
    
    test_df = pd.DataFrame([['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323],
 ['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist'])
    
    test_list = [243, 985]
    test_smiles = ['C1=CC=C(C=C1)C(=O)O', 'CCCCCCCCCCCCCCCC(=O)O']
    
    fingerprint_list = []
    for smile in test_smiles:
        mol = Chem.rdmolfiles.MolFromSmiles(smile)
        fingerprint = FingerprintMols.FingerprintMol(mol)
        fingerprint_list.append(fingerprint)
        
    for mol_fingerprint in fingerprint_list:
        test_result = check_for_known(prom_pos, mol_fingerprint, 0.95)
        
        if test_result is not type(pd.core.frame.DataFrame):
            continue
        else:
            test_result.drop(columns=['Fingerprint', 'Mol'], inplace=True)
            assert test_result == expected_df
    
    return


In [13]:
test_check_for_known()

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykerne

In [ ]:
type(testing)

In [49]:
smile, _ = cid_to_smiles(243)

In [50]:
smile

'C1=CC=C(C=C1)C(=O)O'

In [43]:
test_df = pd.DataFrame([['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323],
 ['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist'])
    

In [46]:
test_df.head()

,enzyme,product,PubChemID,SMILES,Dist
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,1.000000
1,EC-1.14.14.80,CPD-10515,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,0.714718
2,EC-1.14.14.80,PALMITATE,504166,CCCCCCCCCCCCCCCC(=O)[O-],0.714718
3,EC-1.14.14.80,OLEATE-CPD,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],0.714718
4,EC-1.14.14.80,STEARIC_ACID,3033836,CCCCCCCCCCCCCCCCCC(=O)[O-],0.714718


In [ ]:
def cid_to_smiles(cid):

    try:
        compound = pc.get_compounds(cid)[0]
        smiles = compound.canonical_smiles
    except BaseException:
        pass
    
    return smiles, cid


def dist_for_expansion(prom_df, pubchem_cid, num_similar):
    
    #####this probably isn't necessary because in the final version, this will all be run in one sitting and not saved and loaded etc
    #if 'Fingerprint' and 'Mol' in prom_df.columns:
     #   prom_df = prom_df.drop(columns=['Fingerprint','Mol'])
    #else:
     #   continue
    
    bad_cols = ['ExpanDist']
    
    for col in bad_cols:
        if col in prom_df.columns:
            prom_df.drop(columns=col, inplace=True)
        else:
            continue
    
        
    smiles, _ = cid_to_smiles(pubchem_cid)
    
    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    
    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    fingerprint = FingerprintMols.FingerprintMol(mol)
    
    
    prom_df['ExpanDist'] = ''
    
    for index, row in prom_df.iterrows():
        comp = DataStructs.FingerprintSimilarity(row['Fingerprint'], fingerprint, metric=DataStructs.TanimotoSimilarity)
        prom_df['ExpanDist'].loc[index] = comp

    prom_df.sort_values(by='ExpanDist', ascending=False, inplace=True)
    
    if num_similar == 'None':
        n = 20
    else:
        n = num_similar
    
    selected = prom_df.iloc[:n,:].copy()
    
    return selected

In [38]:
%%writefile test_expansion_models.py
import numpy as np
import pandas as pd
import pubchempy as pc

#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

from pandas.util.testing import assert_frame_equal

import expansion_models


def test_check_for_known():
    
    expected_df = pd.DataFrame([['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist', 'Known'])
    
    test_df = pd.DataFrame([['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323],
 ['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'STEARIC_ACID',
  3033836,
  'CCCCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist'])
    
    test_list = [243, 985]
    test_smiles = ['C1=CC=C(C=C1)C(=O)O', 'CCCCCCCCCCCCCCCC(=O)O']
    
    fingerprint_list = []
    for smile in test_smiles:
        mol = Chem.rdmolfiles.MolFromSmiles(smile)
        fingerprint = FingerprintMols.FingerprintMol(mol)
        fingerprint_list.append(fingerprint)
        
    for mol_fingerprint in fingerprint_list:
        test_result = expansion_models.check_for_known(prom_pos, mol_fingerprint, 0.95)
        
        if test_result is not type(pd.core.frame.DataFrame):
            continue
        else:
            test_result.drop(columns=['Fingerprint', 'Mol'], inplace=True)
            assert test_result == expected_df
    
    return



def test_dist_for_expansion():
    test_df = pd.DataFrame([['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323],
 ['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist'])
    
    
    expected = pd.DataFrame([['EC-1.14.14.80',
  'PALMITATE',
  504166,
  'CCCCCCCCCCCCCCCC(=O)[O-]',
  0.714717543728323,
  1.0],
 ['EC-1.14.14.80',
  'OLEATE-CPD',
  5460221,
  'CCCCCCCCC=CCCCCCCCC(=O)[O-]',
  0.714717543728323,
  0.75],
 ['EC-1.14.14.80',
  'CPD-10515',
  25201835,
  'CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O',
  0.714717543728323,
  0.7377049180327869],
 ['EC-1.14.14.77',
  '2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE',
  5280483,
  'CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(C)CCCC(C)C',
  1.0,
  0.3515625]], columns=['enzyme', 'product', 'PubChemID', 'SMILES', 'Dist', 'ExpanDist'])
    
    fingers = expansion_models.fingerprint_products(test_df)
    actual = expansion_models.dist_for_expansion(fingers, 985, 4)
    actual.drop(columns=['Mol', 'Fingerprint'], inplace=True)
    assert_frame_equal(actual.reset_index(drop=True), expected.reset_index(drop=True), check_dtype=False)
    
    return

Overwriting test_expansion_models.py


In [30]:
test_dist_for_expansion()

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
